In [1]:
!pip install xgboost
import xgboost as xgb

In [3]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
# Import dataset
cr = pd.read_csv('./data/cs-df.csv')
cr.shape

(114714, 9)

In [5]:
print(cr.columns)
cr.head()

Index(['seriousdlqin2yrs', 'age_group', 'category_pastdue',
       'dependents_groups', 'rsll_groups', 'ocll_quantile_groups',
       'sqrt_debtratio', 'sqrt_monthlyincome',
       'sqrt_revolvingutilizationofunsecuredlines'],
      dtype='object')


,seriousdlqin2yrs,age_group,category_pastdue,dependents_groups,rsll_groups,ocll_quantile_groups,sqrt_debtratio,sqrt_monthlyincome,sqrt_revolvingutilizationofunsecuredlines
0,1,40-49,Mostly 30-59 Days Past Due,2,3+,11+,0.896093,95.498691,0.875287
1,0,40-49,No Past Due,1,0,0-4,0.349108,50.990195,0.978341
2,0,30-39,Mostly 90 days/+ Past Due,0,0,0-4,0.291742,55.154329,0.811283
3,0,30-39,No Past Due,0,0,5-7,0.189868,57.445626,0.483539
4,0,60+,No Past Due,1,1,0-4,0.612868,59.160798,0.461713


In [20]:
X = cr.iloc[:,1:]
X.dtypes

age_group                                     object
category_pastdue                              object
dependents_groups                             object
rsll_groups                                   object
ocll_quantile_groups                          object
sqrt_debtratio                               float64
sqrt_monthlyincome                           float64
sqrt_revolvingutilizationofunsecuredlines    float64
dtype: object

In [71]:
X['age_group'] = X['age_group'].astype('category')
X['category_pastdue'] = X['category_pastdue'].astype('category')
X['dependents_groups'] = X['dependents_groups'].astype('category')
X['rsll_groups'] = X['rsll_groups'].astype('category')
X['ocll_quantile_groups'] = X['ocll_quantile_groups'].astype('category')
X['sqrt_debtratio'] = X['sqrt_debtratio'].astype('float')
X['sqrt_monthlyincome'] = X['sqrt_monthlyincome'].astype('float')
X['sqrt_revolvingutilizationofunsecuredlines'] = X['sqrt_revolvingutilizationofunsecuredlines'].astype('float')
print(X.dtypes)
y = cr[['seriousdlqin2yrs']]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=3011)

age_group                                    category
category_pastdue                             category
dependents_groups                            category
rsll_groups                                  category
ocll_quantile_groups                         category
sqrt_debtratio                                float64
sqrt_monthlyincome                            float64
sqrt_revolvingutilizationofunsecuredlines     float64
dtype: object


In [26]:
dtrain_classifier = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_classifier = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [29]:
params = {"objective": "multi:softprob", "tree_method": "gpu_hist", "num_class": 5}
n = 1000
results = xgb.cv(
   params, dtrain_classifier,
   num_boost_round=n,
   nfold=5,
   metrics=["mlogloss", "auc", "merror"],
)

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:33:44] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:33:44] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


XGBoostError: [14:33:44] /workspace/src/tree/updater_gpu_hist.cu:781: Exception in gpu_hist: [14:33:44] /workspace/src/tree/updater_gpu_hist.cu:787: Check failed: ctx_->gpu_id >= 0 (-1 vs. 0) : Must have at least one device
Stack trace:
  [bt] (0) /home/codespace/.python/current/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0xb27f2a) [0x7f6826624f2a]
  [bt] (1) /home/codespace/.python/current/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0xb3e95a) [0x7f682663b95a]
  [bt] (2) /home/codespace/.python/current/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0xb483cd) [0x7f68266453cd]
  [bt] (3) /home/codespace/.python/current/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x460c79) [0x7f6825f5dc79]
  [bt] (4) /home/codespace/.python/current/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x461d09) [0x7f6825f5ed09]
  [bt] (5) /home/codespace/.python/current/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x4c54f7) [0x7f6825fc24f7]
  [bt] 

In [31]:
X_train.dtypes

age_group                                    category
category_pastdue                             category
dependents_groups                            category
rsll_groups                                  category
ocll_quantile_groups                         category
sqrt_debtratio                                float64
sqrt_monthlyincome                            float64
sqrt_revolvingutilizationofunsecuredlines     float64
dtype: object

In [48]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
para = {
    'max_depth' : [3],
    'learning_rate' : [0.1],
    'n_estimators' : [100],
    'objective' : ['binary:logistic'],  
    'subsample' : [1],
    'colsample_bytree' : [1],
    'random_state' : [3011]
}
model = xgb.XGBClassifier()
grid_search = GridSearchCV(estimator=model,param_grid=para,cv=5)
grid_search.fit(dtrain_classifier,y_train)


TypeError: Singleton array array(<xgboost.core.DMatrix object at 0x7f6823f2bc70>, dtype=object) cannot be considered a valid collection.

In [76]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder

categorical_columns = ['age_group', 'category_pastdue', 'dependents_groups', 'rsll_groups', 'ocll_quantile_groups']
encoder = OneHotEncoder(handle_unknown='ignore')
# Fit the encoder on the categorical columns and transform the data
X_train_encoded = pd.DataFrame(encoder.fit_transform(X_train[categorical_columns]).toarray())

# Get the unique categories for each categorical column
categories = encoder.categories_

# Construct the column names for the encoded features
encoded_column_names = []
for i, column in enumerate(categorical_columns):
    encoded_column_names.extend([f"{column}_{category}" for category in categories[i]])

# Assign the column names to the encoded DataFrame
X_train_encoded.columns = encoded_column_names

# Concatenate the encoded features with the remaining columns
X_train_encoded = pd.concat([X_train.drop(columns=categorical_columns), X_train_encoded], axis=1)
param_dist = {"n_estimators":sp_randint(40,100),
               "colsample_bytree":np.array([0.5,0.6,0.7,0.8,0.9,1]),
               "subsample":np.array([0.5,0.6,0.7,0.8,0.9,1]),
               "reg_lambda":np.array([1e-5,1e-4,1e-3,1e-2,0.1,1,10,100]),
               "reg_alpha":np.array([1e-5,1e-4,1e-3,1e-2,0.1,1,10,100]),
               "min_child_samples": sp_randint(25,65),
                 "max_depth": sp_randint(1,20)}

model = xgb.XGBClassifier(boosting_type = "gbdt",n_jobs =-1,random_state = 0,verbosity =0,scale_pos_weight = 3.2158)
xgb_random_cv_model = RandomizedSearchCV(model, param_distributions=param_dist,
                                    n_iter=20,cv=5,scoring='roc_auc',random_state=42,verbose=1)

# Fit the RandomizedSearchCV to the data
xgb_random_cv_model.fit(X_train_encoded, y_train)



ValueError: Found input variables with inconsistent numbers of samples: [107434, 86035]

In [62]:
X_train.dtypes

age_group                                    category
category_pastdue                             category
dependents_groups                            category
rsll_groups                                  category
ocll_quantile_groups                         category
sqrt_debtratio                                float64
sqrt_monthlyincome                            float64
sqrt_revolvingutilizationofunsecuredlines     float64
dtype: object